## testing aggregation of time series with tsam
- docs: https://tsam.readthedocs.io/en/latest/index.html
- example notebooks on github: 
  - https://github.com/FZJ-IEK3-VSA/tsam/blob/master/examples/aggregation_example.ipynb
  - https://github.com/FZJ-IEK3-VSA/tsam/blob/master/examples/aggregation_optiinput.ipynb

  

In [ ]:
import pandas as pd
import tsam.timeseriesaggregation as tsam
import plotly.express as px

using PV, Wind Onshore and wind offshore profiles as an example (8760h):

In [ ]:
raw = pd.read_csv("tsam_test_profiles.csv", index_col=0)
raw.plot(subplots=True)

create an aggregation instance and define the settings:

- Most important, define the number of periods, and the duration of each period. 
- TODO: I do not yet understand the other settings (cluster method, representation method)

In [ ]:
aggregation = tsam.TimeSeriesAggregation(
    raw,
    noTypicalPeriods=4,
    hoursPerPeriod=24 * 7,
    representationMethod="distributionAndMinMaxRepresentation",
    distributionPeriodWise=False,
    clusterMethod="hierarchical",
)

perform the aggregation (i.e. define characteristical periods):

In [ ]:
typPeriods = aggregation.createTypicalPeriods()

plot the resulting time series:


In [ ]:
df = typPeriods.reset_index().melt(id_vars=["level_0", "TimeStep"], var_name="RES")
df.head()
px.line(df, x="TimeStep", y="value", facet_col="level_0", facet_row="RES")

You can check how often a certain period appears (this can be used as weighting factor in the objective function of the optimization problem):

In [ ]:
occurence = aggregation.clusterPeriodNoOccur

ax = pd.Series(occurence).plot(kind="bar")
ax.set(ylabel="Number of occurence", xlabel="Period index")
print(pd.Series(occurence).sum())

You can also check the order of apparance:

In [ ]:
df = pd.Series(aggregation.clusterOrder) + 1
df.plot(kind="bar")

and you can get a mapping of new and original index: 

In [ ]:
indexMatching = aggregation.indexMatching()

indexMatching.head(24)

## TODOS for modeling: 
- create test model
- implemente weights of time steps according to how often a period occurs
- create extra condition to enable short term storage inside periods
  - good: SOC in first and last time step must be identical (requires new equation)
  - easy: set SOC to fixed (and identical) value in first and last time step
- find metrics for quality of representation 
- run model with different tsam settings to find a good balance between good representation and fast solve times